In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import random
import datetime

In [16]:
chromedriver = '/Users/daphne/JupyterProjects/新聞＆指數分析/新聞爬蟲/chromedriver3'
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.bbc.co.uk/search?q=trump")

#find click button path
more_path = '//*[@id= "search-content"]/nav[1]/a'

for i in range(35):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.find_element_by_xpath(more_path).click()
    driver.implicitly_wait(5)

pageSource = driver.page_source
bsObj = BeautifulSoup(pageSource)

bsObj

<html class="b-reith-sans-font b-footer--transparent--dark-grey orb-js id-svg b-reith-sans-loaded js hashchange history cssanimations csstransitions video audio localstorage svg no-windows loaded se-no-touch orb-more-loaded" lang="en-GB"><head> <title>BBC - Search results for trump</title> <meta content="BBC, Search" name="keywords"/> <meta content="Application" name="description"/> <meta content="text/html; charset=utf-8" http-equiv="content-type"/> <meta content="width=device-width, initial-scale=1.0" name="viewport"/> <!-- Nav Env: live -->
<!-- Analytics Web Module: 0.0.1-284.0e13ce9 -->
<!-- NavID Web Module: 0.2.0-46.cd5363e -->
<!-- Searchbox Web Module: 2.0.0-17.7e7753b -->
<!-- Promo Web Module: 2.0.0-105.e37c4ea -->
<meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="100004154058350" property="fb:admins"/><link href="https://nav.files.bbci.co.uk/orbit/2.0.0-553.a5d9130a/css/orb-ltr.min.css" rel="stylesheet"/><!--[if (lt IE 9) & (!IEMobile)]>


In [17]:
nodes = bsObj.find_all('article', {'class':"has_image media-video"})
print(nodes)

[<article class="has_image media-video" data-bbc-container="search-query-results" data-bbc-metadata='{"PAR":"search-cards::1","CHD":1,"IMG":"urn:bbc:imagepid:p06qgvdr"}' data-bbc-personalisation='{"EDN":"default","QRY":"trump"}' data-bbc-source="Programmes | BBC One" data-bbc-title="Panorama: Trump: What Next?" itemscope="">
<aside class="flags top">
<dl>
<dt>Published Date</dt>
<dd>
<time class="display-date" datetime="2018-11-05T00:00:00Z">
           5 Nov 2018       </time>
</dd>
</dl>
</aside>
<a href="https://www.bbc.co.uk/programmes/b0brg0ls" id="search-result-0">
<picture>
<img alt="" class="imagepid" src="https://ichef.bbci.co.uk/images/ic/192xn/p06qgvdr.jpg" srcset="https://ichef.bbci.co.uk/images/ic/192xn/p06qgvdr.jpg, https://ichef.bbci.co.uk/images/ic/384xn/p06qgvdr.jpg 2x"/> </picture>
</a>
<aside class="flags mid">
<dl>
<dt>Published Date</dt>
<dd>
<time class="display-date" datetime="2018-11-05T00:00:00Z">
           5 Nov 2018       </time>
</dd>
</dl>
</aside>
<div>
<

In [55]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
proxies = ""

def get_doc(href):
    resp = requests.get(href, headers=headers, proxies=proxies)
    resp.encoding = 'utf-8'
    bsObj_news = BeautifulSoup(resp.content, "lxml")
    a_news = bsObj_news.find_all('p', class_="")
    docs = []
    for d in a_news:
        doc = d.get_text()
        docs.append(doc)
    
    a_doc = ''
    for doc in docs:
        all_doc = a_doc + doc
        a_doc = all_doc 
    
    return all_doc

'''html = 'https://www.bbc.co.uk/news/world-us-canada-47592271'
news_doc = get_doc(html)
print(news_doc)'''


President Donald Trump has told reporters he did not think white nationalism was a growing threat following the mosque shootings in New Zealand. The president instead suggested white nationalists were "a small group of people".


In [56]:
news = []
for node in nodes:
    title = node.find('h1', {'itemprop': "headline"}).get_text()
    summary = node.find('p', {'class': "summary long"}).get_text()
    href = node.find('a').get('href')
    doc = get_doc(href)
    try:
        time = node.find('time', {'class': "display-date"}).get_text().strip()
        #print(type(time))  #str
    except AttributeError:
        time = None
    #href = node.find({'a': 'href'})
    #print(time)
    #print(title)
    #print(href)
    #print(summary)
    #print(doc)
    #print("\n")
    if time != None:
        node_dict = dict()
        node_dict['time'] = time
        node_dict['title'] = title
        node_dict['summary'] = summary
        node_dict['doc'] = doc
        news.append(node_dict)
    else:
        pass

print(len(nodes))   #123

123


In [57]:
news   #len: 119

[{'time': '5 Nov 2018',
  'title': 'Panorama: Trump: What Next?',
  'summary': "…Will President Trump be forced out of office? On the eve of the most important… US midterm elections for a generation, Panorama examines allegations that Trump… tomorrow, it could open the door to impeachment, but Trump's supporters believe…",
  'doc': "On the eve of the most important US midterm elections for a generation, Panorama examines allegations that Trump colluded with Russia to win the presidency.Will President Trump be forced out of office? On the eve of the most important US midterm elections for a generation, Panorama examines allegations that Trump colluded with Russia to win the presidency, and looks at the similarities with the Watergate scandal which brought down Richard Nixon. Reporter Hilary Andersson joins the campaign trail in West Virginia, where the president's message of a vast political witch-hunt against him has fired up Republican voters. If the Democrats do well tomorrow, it cou

In [59]:
#存取資料
import pickle
fh = open('bbc_trump', 'wb')
pickle.dump(news, fh)
fh.close()